In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(
    "nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True
)
model = AutoModel.from_pretrained(
    "nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
inputs = tokenizer(
        ["ABC", "DEF"],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,
    )
inputs = {k: v.to(device) for k, v in inputs.items()}


with torch.no_grad():
    outputs = model(**inputs)
    # Use the mean pooling of the last hidden state as embedding
    last_hidden = outputs.last_hidden_state
    attention_mask = inputs["attention_mask"]
    mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
    sum_embeddings = torch.sum(last_hidden * mask_expanded, 1)
    sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
    embeddings = (sum_embeddings / sum_mask).squeeze().tolist()

In [ ]:
# embeddings[1]
